# Export data to files with Suitcase

In this notebook you will:

* Export saved data to CSV and TIFF.
* See a preview of how exporting data live during a scan works.

## Configuration
Below, we will connect to EPICS IOC(s) controlling simulated hardware in lieu of actual motors, detectors. The IOCs should already be running in the background. Run this command to verify that they are running: it should produce output with RUNNING on each line. In the event of a problem, edit this command to replace `status` with `restart all` and run again.

In [1]:
!supervisorctl -c supervisor/supervisord.conf status

decay                            RUNNING   pid 4900, uptime 0:00:33
mini_beamline                    RUNNING   pid 4901, uptime 0:00:33
random_walk                      RUNNING   pid 4902, uptime 0:00:33
random_walk_horiz                RUNNING   pid 4903, uptime 0:00:33
random_walk_vert                 RUNNING   pid 4904, uptime 0:00:33
simple                           RUNNING   pid 4905, uptime 0:00:33
thermo_sim                       RUNNING   pid 4906, uptime 0:00:33
trigger_with_pc                  RUNNING   pid 4907, uptime 0:00:33


In [2]:
%run scripts/beamline_configuration.py

## Scalar data to CSV

In [3]:
RE(scan([ph], motor_ph, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 3     Time: 2020-05-19 00:52:27
Persistent Unique Scan ID: 'cd804db0-780d-451d-8165-714c11b12e99'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+------------+
|   seq_num |       time |   motor_ph |     ph_det |
+-----------+------------+------------+------------+
|         1 | 00:52:28.0 |     -1.000 |      97955 |


|         2 | 00:52:28.0 |     -0.500 |      97468 |
|         3 | 00:52:28.0 |      0.000 |      98258 |
|         4 | 00:52:28.0 |      0.500 |      98008 |
|         5 | 00:52:28.1 |      1.000 |      95500 |
+-----------+------------+------------+------------+


generator scan ['cd804db0'] (scan num: 3)





('cd804db0-780d-451d-8165-714c11b12e99',)

In [4]:
import suitcase.csv

In [5]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/cd804db0-780d-451d-8165-714c11b12e99-primary.csv')]}

In [6]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}-{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni-12345-primary.csv')]}

In [7]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary.csv')]}

## Image data to TIFF

In [8]:
RE(scan([spot], mtr_spotx, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 4     Time: 2020-05-19 00:52:28
Persistent Unique Scan ID: '703b72e8-6cb0-440f-b02a-ed4969f52ac9'


New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+-------------+------------+
|   seq_num |       time | motor_spotx |   spot_roi |
+-----------+------------+-------------+------------+
|         1 | 00:52:28.5 |          -1 | 1059404389 |


|         2 | 00:52:28.7 |          -0 | 1056590557 |


|         3 | 00:52:28.8 |           0 | 1053771034 |
|         4 | 00:52:28.9 |           0 | 1053340258 |


|         5 | 00:52:29.0 |           1 | 1054127872 |


+-----------+------------+-------------+------------+
generator scan ['703b72e8'] (scan num: 4)





('703b72e8-6cb0-440f-b02a-ed4969f52ac9',)

### TIFF stack, a 3D block of images in one file

In [9]:
import suitcase.tiff_stack

In [10]:
suitcase.tiff_stack.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary-spot_img.tiff')]}

### TIFF series

In [11]:
import suitcase.tiff_series

Unlike CSV or TIFF stack, this exporter creates separate file per row (Event document). We can include information from a specific Event in the filename. Here, we include the motor position.

In [12]:
suitcase.tiff_series.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}/{event[data][motor_spotx]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-1.0-primary-spot_img-0.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-0.5-primary-spot_img-1.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.0-primary-spot_img-2.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.5-primary-spot_img-3.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/1.0-primary-spot_img-4.tiff')]}

## "Beta" feature: Set up live exporting

*We plan to streamline this in the future, but here is a demonstration that it is possible.*

In [13]:
# Execute this once to configure RE to send data to an exporter.
# At the beamline, this would normally be done in a script automatically at startup.

from event_model import RunRouter
from databroker.core import discover_handlers

def factory(name, start_doc):

    serializer = suitcase.tiff_series.Serializer(
        'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-{start[uid]:.8}-')

    def cb(name, doc):
        serializer(name, doc)

    return [cb], []


rr = RunRouter([factory], discover_handlers())
RE.subscribe(rr)

2

In [14]:
RE(count([spot], num=30, delay=1), sample_comp='Lab6', barcode='12345')



Transient Scan ID: 5     Time: 2020-05-19 00:52:29
Persistent Unique Scan ID: 'ab12dd36-be21-4d37-b7b3-a0a63b97b184'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+
|   seq_num |       time |   spot_roi |
+-----------+------------+------------+
|         1 | 00:52:29.5 | 1066401399 |


|         2 | 00:52:30.4 | 1123280746 |


|         3 | 00:52:31.4 | 1116728980 |


...

|        28 | 00:52:56.4 | 1063139884 |


|        29 | 00:52:57.4 | 1070170106 |


|        30 | 00:52:58.4 | 1124994213 |


+-----------+------------+------------+
generator count ['ab12dd36'] (scan num: 5)





('ab12dd36-be21-4d37-b7b3-a0a63b97b184',)